In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import vgg16
from torch import nn, optim
import torch.nn.functional as F

# Define your YOLOv8 model architecture
class YOLOv8(nn.Module):
    def __init__(self, num_classes):
        super(YOLOv8, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.fc = nn.Linear(1024 * 7 * 7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 1024 * 7 * 7)
        x = self.fc(x)
        return x

# Define custom dataset class for blood group detection
class BloodGroupDataset(Dataset):
    def __init__(self, image_folder, label_folder, transform=None):
        self.images = datasets.ImageFolder(image_folder, transform=transform)
        self.labels = datasets.ImageFolder(label_folder, transform=transform)
        # Implement logic to load labels here
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image, _ = self.images[idx]
        label, _ = self.labels[idx]
        # Implement logic to load images and corresponding labels here
        return image, label


# Define data transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define paths to your data
train_data_path = "/Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/train/images"
train_label_path = "/Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/train/labels"

test_data_path = "/Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/test/images"
test_label_path = "/Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/test/labels"

valid_data_path = "/Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/valid/images"
valid_label_path = "/Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/valid/labels"

# Create datasets and dataloaders
train_dataset = BloodGroupDataset(train_data_path, train_label_path, transform=transform)
test_dataset = BloodGroupDataset(test_data_path, test_label_path, transform=transform)
valid_dataset = BloodGroupDataset(valid_data_path, valid_label_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Initialize your YOLOv8 model
model = YOLOv8(num_classes=...)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print training loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluate the model on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy}")


FileNotFoundError: Couldn't find any class folder in /Users/mohammadadnaan/Desktop/Projects/BloodGroupDetection/train/images.